In [7]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
import pandas as pd
import numpy as np

In [8]:
def file_load(txt_path):
    
    data_path = []    
    f = open(txt_path , 'r')
    while True:
        line = f.readline()
        if not line: break
        data_path.append(line[:-1])
    f.close()
    return data_path

def emg_sampler(emg,target_length):
    
    sampled_emg = emg[::4]
    if len(sampled_emg) >= target_length:
        return sampled_emg[:target_length]
    else:
        output = np.zeros((target_length,8))
        output[:len(sampled_emg)] = sampled_emg
        return output
    
    

In [9]:
class Myodata(Dataset):
    def __init__(self, train=True):
        super().__init__()

        self.max_len = 869
        """
        opt_data : 'train', 'validation'
        
        """
        if train == True:
            self.file_list = file_load('train.txt')
        else:
            self.file_list = file_load('test.txt')
                    
    def __getitem__(self, index):
        
        info = self.file_list[index].split(' ')
        acc = pd.read_csv(info[1],header=None).values
        gyr = pd.read_csv(info[2],header=None).values
        emg = pd.read_csv(info[0],header=None).values
        emg = emg_sampler(emg,len(acc))
        
        x = np.concatenate((emg,acc,gyr),axis=1)
        pad_len = self.max_len - len(x)
        try:
            self.x_data = torch.tensor(np.pad(x,pad_width=((pad_len,0), (0, 0)))).T
        except:
            print('file error',info)
        self.y_data = torch.tensor(int(info[3])).float()
        return self.x_data, self.y_data

    def __len__(self):
        return len(self.file_list)



In [10]:
data = Myodata()

In [11]:
i = 0

In [12]:
for x,y in data:
    i += 1
    if i % 100 == 0:
        print('hi')

hi
hi
hi
hi
hi
hi
hi
file error ['data/myo/P059/emg/A043_P059_G004.csv', 'data/myo/P059/accel/A043_P059_G004.csv', 'data/myo/P059/gyro/A043_P059_G004.csv', '43']
hi
hi
hi
hi
hi
file error ['data/myo/P064/emg/A025_P064_G001.csv', 'data/myo/P064/accel/A025_P064_G001.csv', 'data/myo/P064/gyro/A025_P064_G001.csv', '25']
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi


ValueError: could not broadcast input array from shape (92,11) into shape (92,8)

In [ ]:
train_loader = DataLoader(dataset = data, batch_size= 64, shuffle=True )

In [ ]:
for x,y in train_loader:
    print(x.shape)